In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import uniform, randint
import xgboost as xgb

# Load the dataset
file_path = 'Transition_WACC_And_Price.xlsx'  # Replace with the path to your file
data = pd.read_excel(file_path, sheet_name='Sheet1')

# Define energy types and their real values for 2018 (in percentage form)
energy_types = [ 'Nuclear', 'Hydro', 'Biomass', 'CCS', 'Solar', 'Onshore', 'Offshore']
real_values_2018 = [ 5.2, 4.5, 5.6, 5.6, 5.1, 4.8, 6.3]

# Convert real values to decimal format (e.g., 5.6% -> 0.056)
real_values_2018 = [value / 100 for value in real_values_2018]

# Define features for energy prices
features = {
    'Oil_Price': data[['Oil_Price']],
    'Coal_Price': data[['Coal_Price']],
    'Gas_Price': data[['Gas_Price']]
}

# Normalize the feature data
scaler = StandardScaler()
for key in features:
    features[key] = pd.DataFrame(scaler.fit_transform(features[key]), columns=features[key].columns)

# Define base learners for the stacking regressor
base_learners = [
    ('rf', RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42)),  # Random forest with increased trees and depth
    ('xgb', xgb.XGBRegressor(n_estimators=300, max_depth=12, learning_rate=0.03, random_state=42))  # XGBoost with fine-tuned parameters
]

# Initialize the stacking regressor
stack_regressor = StackingRegressor(estimators=base_learners, final_estimator=LinearRegression())

# Define hyperparameter search space for RandomizedSearchCV
param_distributions = {
    'rf__n_estimators': randint(200, 400),          # Random forest: number of trees
    'rf__max_depth': randint(10, 20),               # Random forest: depth of trees
    'xgb__n_estimators': randint(200, 400),         # XGBoost: number of trees
    'xgb__learning_rate': uniform(0.01, 0.05),      # XGBoost: learning rate
    'xgb__max_depth': randint(10, 15)               # XGBoost: depth of trees
}

# Initialize a list to store results
results = []

# Loop through each type of price feature
for feature_name, feature_data in features.items():
    print(f"\n--- Using {feature_name} as the feature ---\n")
    predicted_values_2018 = []

    # Predict WACC for each energy type
    for energy_index, energy in enumerate(energy_types):
        print(f"Processing energy type: {energy}")
        y = data[energy]  # Target variable

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(feature_data, y, test_size=0.2, random_state=42)

        # Check if there is enough data to train and test
        if len(X_train) < 2 or len(y_train) < 2:
            print(f"Skipping {energy} due to insufficient data.")
            continue

        # Use RandomizedSearchCV for hyperparameter tuning
        random_search = RandomizedSearchCV(
            estimator=stack_regressor,
            param_distributions=param_distributions,
            n_iter=50,        # Increase random search iterations
            cv=5,             # Use 5-fold cross-validation for robustness
            scoring='neg_mean_absolute_error',  # Optimize for MAE
            n_jobs=-1,        # Use all available CPU cores
            random_state=42
        )
        random_search.fit(X_train, y_train)

        # Use the best model to predict the WACC for 2018
        if feature_name == 'Oil_Price':
            price_2018 = pd.DataFrame([[68.34]], columns=['Oil_Price'])  # Wrap the value in a DataFrame with the correct column name
        elif feature_name == 'Coal_Price':
            price_2018 = pd.DataFrame([[102.36]], columns=['Coal_Price'])  # Wrap the value in a DataFrame with the correct column name
        elif feature_name == 'Gas_Price':
            price_2018 = pd.DataFrame([[7.17]], columns=['Gas_Price'])  # Wrap the value in a DataFrame with the correct column name

        predicted_value = random_search.best_estimator_.predict(price_2018)[0]
        predicted_values_2018.append(predicted_value)

    # Ensure that the number of predicted and real values matches
    if len(predicted_values_2018) == len(real_values_2018):
        # Store the predicted and real values in the results
        for energy_index, (predicted, real) in enumerate(zip(predicted_values_2018, real_values_2018)):
            results.append({
                'Feature': feature_name,
                'Energy Type': energy_types[energy_index],
                'Predicted': predicted,
                'Real': real,
                'Error': abs(predicted - real)  # Calculate absolute error for each prediction
            })

        # Calculate RMSE and MAE
        rmse = np.sqrt(mean_squared_error(real_values_2018, predicted_values_2018))
        mae = mean_absolute_error(real_values_2018, predicted_values_2018)

        # Print RMSE and MAE
        print(f"\nRMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
    else:
        print(f"Skipping final evaluation for {feature_name} due to mismatched data sizes.")

# Convert results to a DataFrame for easier visualization
results_df = pd.DataFrame(results)
print("\n--- Final Results ---\n")
print(results_df)

# Optionally save the results to a file
results_df.to_excel('Predicted_vs_Real_Results_Improved.xlsx', index=False)



--- Using Oil_Price as the feature ---

Processing energy type: Coal
Processing energy type: Gas
Processing energy type: Nuclear
Processing energy type: Hydro
Processing energy type: Biomass
Processing energy type: CCS
Processing energy type: Solar
Processing energy type: Onshore
Processing energy type: Offshore

RMSE: 0.0079
MAE: 0.0071

--- Using Coal_Price as the feature ---

Processing energy type: Coal
Processing energy type: Gas
Processing energy type: Nuclear
Processing energy type: Hydro
Processing energy type: Biomass
Processing energy type: CCS
Processing energy type: Solar
Processing energy type: Onshore
Processing energy type: Offshore

RMSE: 0.0083
MAE: 0.0074

--- Using Gas_Price as the feature ---

Processing energy type: Coal
Processing energy type: Gas
Processing energy type: Nuclear
Processing energy type: Hydro
Processing energy type: Biomass
Processing energy type: CCS
Processing energy type: Solar
Processing energy type: Onshore
Processing energy type: Offshore

R